In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare
import os
import subprocess
import tensorflow as tf
from pdf2image import convert_from_path, convert_from_bytes
from tkinter import filedialog as fd


In [51]:
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [89]:
weight_path = fd.askopenfilename()
model = torch.hub.load('yolov5', 'custom', path=weight_path, source='local')
# !python yolov5/export.py --weights yolob5s.pt

YOLOv5  v6.0-266-g07221f1 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [79]:
model_onnx = onnx.load('yolov5s.onnx')
tf_rep = prepare(model_onnx)
tf_rep.export_graph('modelTF')


Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: modelTF\assets


Assets written to: modelTF\assets


SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.


In [101]:
pb_modle = tf.saved_model.load(r'modelTF')
f = pb_modle.signatures["serving_default"]

In [90]:
filepath = fd.askopenfilename()
filename_and_ext = os.path.basename(filepath).split('.')

In [93]:
threshold = 0.1

In [ ]:
images = convert_from_path(filepath, dpi=95)
results = []
for i in range(len(images)):
    k = 1    
    saved_filename = filename_and_ext[0] + '_' + str(i+1)
    images[i].save( saved_filename + '.png', 'PNG')

    
    result_deteced_from_image = model(saved_filename + '.png')
    # result_deteced_from_image.save(saved_filename+'_from model' + '.png')
    results.append(result_deteced_from_image)
    data_detected = results[i].pandas().xyxy[0]    
    print(data_detected)
    num_object = data_detected.shape[0]
    for j in range(num_object):
        if data_detected['confidence'][j] > threshold:
            imc = images[i].crop((data_detected['xmin'][j], data_detected['ymin'][j], data_detected['xmax'][j], data_detected['ymax'][j]))    
            image_name = str(data_detected['confidence'][j]) + '_' + str(k) + '_' + filename_and_ext[0] + '_' + str(i+1)            
            imc.save(image_name+'.png')
            k += 1    

In [176]:
from PIL import Image
img = Image.open('test_2.png')
img = img.convert('RGB')
img.size


(808, 1045)

In [204]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
test_image = load_img('test_2.png', target_size=(640,640))
# print(test_image)
image =  img_to_array(test_image)/255.0
image = np.transpose(image)

# print(image)
import numpy as np
image = np.expand_dims(image, axis=0)


In [205]:
image.shape

(1, 3, 640, 640)

In [172]:
converted_imgage = tf.image.convert_image_dtype(img, dtype='float')

In [173]:
# tt = tf.expand_dims(converted_imgage, axis=0)
tt = converted_imgage


In [174]:
print(tt.shape)

(1045, 808, 3)


In [217]:
pb_modle = tf.saved_model.load(r'modelTF')
f = pb_modle.signatures["serving_default"]
f.output_shapes


{'output': TensorShape([1, 25200, 85]),
 '339': TensorShape([1, 3, 80, 80, 85]),
 '443': TensorShape([1, 3, 20, 20, 85]),
 '391': TensorShape([1, 3, 40, 40, 85])}

In [216]:
print(image.shape)
print(type(image))
image = tf.convert_to_tensor(image, dtype='float')
outputs = f(image)
outputs['output']


(1, 3, 640, 640)
<class 'tensorflow.python.framework.ops.EagerTensor'>


<tf.Tensor: shape=(1, 25200, 85), dtype=float32, numpy=
array([[[     4.9628,      2.0368,       9.751, ...,   0.0022384,   0.0020137,    0.031211],
        [     11.478,      2.3212,      20.426, ...,    0.002461,   0.0022058,     0.02976],
        [     18.209,      2.1291,      26.646, ...,   0.0021883,   0.0025875,    0.045289],
        ...,
        [      565.8,      606.15,      156.95, ...,   0.0058575,   0.0023872,    0.013404],
        [     586.22,      607.89,      123.86, ...,   0.0084011,   0.0026894,    0.021166],
        [     621.59,      618.72,       130.1, ...,   0.0076936,   0.0026491,    0.014792]]], dtype=float32)>